In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### State

In [2]:
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]

### 상담사 chatbot

In [7]:
from typing import List

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.prompts import MessagesPlaceholder

from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
from langchain_core.messages import BaseMessage


def call_chatbot(messages: List[BaseMessage]) -> dict:
    print('==================================================================================================')
    print(f'🔄 [Node] 상담사 chatbot 노드 🔄')
    
    print(f"고객 질문: ")

    for msg in messages:
        print(msg)


    prompt = ChatPromptTemplate.from_messages(
        [
            (
                'system',
                'You are a customer support agent for an airline. Answer in Korean.'
            ),
            MessagesPlaceholder(variable_name='messages') 
        ]        
    )

    llm = ChatOpenAI(
        api_key=key, 
        model='gpt-4o-mini', 
        temperature=0.6
    )

    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"messages": messages})
    
    print(f"상담사 chatbot 답변: ")
    print(answer)
    
    print('==================================================================================================')

    return answer

In [11]:
call_chatbot([('user', '안녕하세요?')])

# message = HumanMessage(content="안녕하세요? 저 화가 많이 났어요")
# call_chatbot([message])

🔄 [Node] 상담사 chatbot 노드 🔄
고객 질문: 
('user', '안녕하세요?')
상담사 chatbot 답변: 
안녕하세요! 어떻게 도와드릴까요?


'안녕하세요! 어떻게 도와드릴까요?'

### 고객이 사용할 시나리오

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.prompts import MessagesPlaceholder


def create_scenario(name: str, instructions: str):
    system_prompt_template = """You are a customer of an airline company. \
You are interacting with a user who is a customer support person. \

Your name is {name}.

# Instructions:
{instructions}

[IMPORTANT] 
- When you are finished with the conversation, respond with a single word 'FINISHED'
- You must speak in Korean."""


    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt_template),
            MessagesPlaceholder(variable_name="messages")
        ]
    )

    prompt = prompt.partial(name=name, instructions=instructions)

    return prompt

In [15]:
instructions = """You are trying to get a refund for the trip you took to Jeju Island. \
You want them to give you ALL the money back. This trip happened last year."""

name = '홍길동'

# create_scenario(name, instructions).pretty_print()

llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0.6
)

simulated_user = create_scenario(name, instructions) | llm | StrOutputParser()

In [16]:
messages = [HumanMessage(content='안녕하세요? 무엇을 도와 드릴까요?')]

simulated_user.invoke({'messages': messages})

'안녕하세요. 저는 작년에 제주도로 다녀온 여행에 대한 환불을 요청하고 싶습니다. 전체 금액을 돌려받고 싶습니다. 어떻게 진행해야 하나요?'